In [40]:
import pandas as pd


df = pd.read_csv("./merged_csv/fin/방문지_fin.csv")

In [46]:
df[df['NEW_VISIT_AREA_ID'] == 39227]

,VISIT_AREA_ID,TRAVEL_ID,VISIT_ORDER,VISIT_AREA_NM,VISIT_START_YMD,VISIT_END_YMD,ROAD_NM_ADDR,LOTNO_ADDR,X_COORD,Y_COORD,...,RESIDENCE_TIME_MIN,VISIT_AREA_TYPE_CD,REVISIT_YN,VISIT_CHC_REASON_CD,LODGING_TYPE_CD,DGSTFN,REVISIT_INTENTION,RCMDTN_INTENTION,SGG_CD,NEW_VISIT_AREA_ID
54952,2309090011,f_f011248,11.0,카페 캐트시,2023-09-09,2023-09-09,강원특별자치도 강릉시 금성로13번길 19-1,강원특별자치도 강릉시 성남동 51-108,128.900232,37.753203,...,60.0,11.0,N,9.0,NaN,2.0,2.0,2.0,NaN,39227
57251,2309080014,f_f013171,14.0,카페 캐트시,2023-09-08,2023-09-08,강원특별자치도 강릉시 금성로13번길 19-1,강원특별자치도 강릉시 성남동 51-108,128.900232,37.753203,...,60.0,11.0,N,2.0,NaN,5.0,5.0,5.0,NaN,39227


In [33]:
df.groupby('NEW_VISIT_AREA_ID')['NEW_VISIT_AREA_ID'].count().sort_values()

NEW_VISIT_AREA_ID
1            1
22280        1
38638        1
22278        1
22277        1
         ...  
20760      377
19263      496
5924       888
34947     1937
2500     22335
Name: NEW_VISIT_AREA_ID, Length: 47130, dtype: int64

In [31]:
import pandas as pd
import numpy as np
import os

def create_visit_area_mapping(df_visit, df_photo=None):
    """방문지 정보에서 장소명 기반으로 NEW_VISIT_AREA_ID 매핑 생성"""
    print("=== NEW_VISIT_AREA_ID 매핑 생성 ===")
    
    # 데이터 복사
    df = df_visit.copy()
    
    # 관광사진에만 있는 데이터 추가
    if df_photo is not None:
        # 방문지에 없는 VISIT_AREA_NM을 가진 사진 데이터 찾기
        visit_names = set(df['VISIT_AREA_NM'].dropna())
        photo_only_names = df_photo[~df_photo['VISIT_AREA_NM'].isin(visit_names) & df_photo['VISIT_AREA_NM'].notna()]
        
        if not photo_only_names.empty:
            # 중복 제거하여 추가
            photo_unique = photo_only_names[['VISIT_AREA_ID', 'VISIT_AREA_NM']].drop_duplicates(subset=['VISIT_AREA_NM'])
            for _, row in photo_unique.iterrows():
                new_row = pd.Series({
                    'VISIT_AREA_ID': row['VISIT_AREA_ID'],
                    'VISIT_AREA_NM': row['VISIT_AREA_NM'],
                    'TRAVEL_ID': '',
                    'ROAD_NM_ADDR': '',
                    'LOTNO_ADDR': '',
                    'X_COORD': None,
                    'Y_COORD': None
                })
                df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    
    def is_residence(name):
        """거주지 판별 - '집'만 체크 (횟집 등 제외)"""
        if not name or pd.isna(name):
            return False
        
        name = str(name).strip()
        
        # 횟집, 술집 등은 제외
        exclude_patterns = ['횟집', '술집', '찜질', '커피', '볶는', '게스트']
        for pattern in exclude_patterns:
            if pattern in name:
                return False
        
        # '집' 관련 패턴
        residence_patterns = [
            name == '집',
            name.endswith(' 집'),
            name.endswith('집)'),
            '부모' in name and '집' in name,
            '친척' in name and '집' in name,
            '우리집' in name,
            name == '본가',
        ]
        
        return any(residence_patterns)
    
    # 장소명별 그룹화
    unique_location_names = set()
    
    for _, row in df.iterrows():
        name = str(row['VISIT_AREA_NM']).strip() if pd.notna(row['VISIT_AREA_NM']) else f"이름없음_ID{row['VISIT_AREA_ID']}"
        
        # 거주지 통합
        if is_residence(name):
            normalized_name = "거주지(집_통합)"
        else:
            normalized_name = name
            
        unique_location_names.add(normalized_name)
    
    # NEW_VISIT_AREA_ID 할당 (1부터 시작)
    name_to_new_id = {}
    new_id = 1
    
    # 정렬된 장소명 순서로 ID 할당
    for location_name in sorted(unique_location_names):
        name_to_new_id[location_name] = new_id
        print(f"NEW_ID {new_id}: '{location_name}'")
        new_id += 1
    
    print(f"\n총 {len(unique_location_names)}개의 고유 장소명을 {new_id-1}개 NEW_ID로 매핑")
    
    # DataFrame에 NEW_VISIT_AREA_ID 컬럼 추가
    def get_new_id(row):
        name = str(row['VISIT_AREA_NM']).strip() if pd.notna(row['VISIT_AREA_NM']) else f"이름없음_ID{row['VISIT_AREA_ID']}"
        
        if is_residence(name):
            normalized_name = "거주지(집_통합)"
        else:
            normalized_name = name
            
        return name_to_new_id.get(normalized_name, None)
    
    df['NEW_VISIT_AREA_ID'] = df.apply(get_new_id, axis=1)
    
    # 매핑이 안된 행이 있는지 확인
    unmapped = df[df['NEW_VISIT_AREA_ID'].isna()]
    if not unmapped.empty:
        print(f"경고: 매핑되지 않은 행이 {len(unmapped)}개 있습니다:")
        print(unmapped[['VISIT_AREA_ID', 'VISIT_AREA_NM']].head())
    
    return df, name_to_new_id

def apply_mapping_to_other_datasets(df_visit, df_photo, df_move, name_to_new_id):
    """다른 데이터셋에 매핑 적용"""
    
    def is_residence(name):
        """거주지 판별 함수 (동일)"""
        if not name or pd.isna(name):
            return False
        
        name = str(name).strip()
        exclude_patterns = ['횟집', '술집', '찜질', '커피', '볶는', '게스트']
        for pattern in exclude_patterns:
            if pattern in name:
                return False
        
        residence_patterns = [
            name == '집',
            name.endswith('집)'),
            '부모' in name and '집' in name,
            '친척' in name and '집' in name,
            '우리집' in name,
            name == '본가',
        ]
        
        return any(residence_patterns)
    
    def get_new_id_from_name(visit_area_nm):
        """장소명으로부터 NEW_VISIT_AREA_ID 가져오기"""
        if pd.isna(visit_area_nm):
            return None
            
        name = str(visit_area_nm).strip()
        
        if is_residence(name):
            normalized_name = "거주지(집_통합)"
        else:
            normalized_name = name
            
        return name_to_new_id.get(normalized_name, None)
    
    # 관광사진 데이터 업데이트
    df_photo_updated = df_photo.copy()
    df_photo_updated['NEW_VISIT_AREA_ID'] = df_photo_updated['VISIT_AREA_NM'].apply(get_new_id_from_name)
    
    # 이동내역 데이터 업데이트를 위해 VISIT_AREA_ID -> VISIT_AREA_NM 매핑 필요
    # 방문지와 관광사진 데이터를 합쳐서 ID -> 이름 매핑 테이블 생성
    id_to_name_mapping = {}
    
    # 방문지 데이터에서 ID -> 이름 매핑
    for _, row in df_visit.iterrows():
        if pd.notna(row['VISIT_AREA_NM']):
            id_to_name_mapping[row['VISIT_AREA_ID']] = row['VISIT_AREA_NM']
    
    # 관광사진 데이터에서 추가 매핑 (기존에 없는 것만)
    for _, row in df_photo.iterrows():
        if pd.notna(row['VISIT_AREA_NM']) and row['VISIT_AREA_ID'] not in id_to_name_mapping:
            id_to_name_mapping[row['VISIT_AREA_ID']] = row['VISIT_AREA_NM']
    
    # 이동내역 데이터 업데이트
    df_move_updated = df_move.copy()
    
    # START_VISIT_AREA_ID -> 이름 -> NEW_ID
    df_move_updated['START_VISIT_AREA_NM'] = df_move_updated['START_VISIT_AREA_ID'].map(id_to_name_mapping)
    df_move_updated['NEW_START_VISIT_AREA_ID'] = df_move_updated['START_VISIT_AREA_NM'].apply(get_new_id_from_name)
    
    # END_VISIT_AREA_ID -> 이름 -> NEW_ID  
    df_move_updated['END_VISIT_AREA_NM'] = df_move_updated['END_VISIT_AREA_ID'].map(id_to_name_mapping)
    df_move_updated['NEW_END_VISIT_AREA_ID'] = df_move_updated['END_VISIT_AREA_NM'].apply(get_new_id_from_name)
    
    return df_photo_updated, df_move_updated

def verify_mapping(df):
    """매핑 결과 검증"""
    print("\n=== 매핑 결과 검증 ===")
    
    # NEW_VISIT_AREA_ID별로 그룹화하여 장소명 확인
    new_id_groups = df.groupby('NEW_VISIT_AREA_ID')['VISIT_AREA_NM'].unique()
    
    print("NEW_VISIT_AREA_ID별 장소명 확인 (처음 10개):")
    for new_id, names in new_id_groups.head(10).items():
        unique_names = [name for name in names if pd.notna(name)]
        print(f"NEW_ID {new_id}: {unique_names}")
        
        # 거주지가 아닌 경우 장소명이 여러개면 문제
        if len(set(unique_names)) > 1:
            # 모두 거주지 관련인지 확인
            all_residence = all('집' in str(name) or '본가' in str(name) for name in unique_names)
            if not all_residence:
                print(f"  ⚠️ 경고: 서로 다른 장소명이 같은 NEW_ID를 가짐")
    
    print(f"\n총 {len(new_id_groups)}개의 서로 다른 NEW_VISIT_AREA_ID 생성")
    print(f"NEW_VISIT_AREA_ID 범위: {df['NEW_VISIT_AREA_ID'].min()} ~ {df['NEW_VISIT_AREA_ID'].max()}")

def main():
    """메인 실행 함수"""
    try:
        # 데이터 로드
        print("=== 데이터 로드 ===")
        df_visit = pd.read_csv('./merged_csv/방문지_total.csv')
        df_photo = pd.read_csv('./merged_csv/관광사진_total.csv')
        df_move = pd.read_csv('./merged_csv/fin/이동내역_fin.csv')
        
        print(f"방문지: {len(df_visit)}행, 관광사진: {len(df_photo)}행, 이동내역: {len(df_move)}행")
        
        # NEW_VISIT_AREA_ID 매핑 생성 (장소명 기준)
        df_visit_updated, name_to_new_id = create_visit_area_mapping(df_visit, df_photo)
        
        # 매핑 결과 검증
        verify_mapping(df_visit_updated)
        
        # 다른 데이터셋에 매핑 적용
        df_photo_updated, df_move_updated = apply_mapping_to_other_datasets(df_visit, df_photo, df_move, name_to_new_id)
        
        # 결과 저장
        print("\n=== 결과 저장 ===")
        os.makedirs('./merged_csv/fin', exist_ok=True)
        
        df_visit_updated.to_csv('./merged_csv/fin/방문지_fin.csv', index=False, encoding='utf-8-sig')
        df_photo_updated.to_csv('./merged_csv/fin/관광사진_fin.csv', index=False, encoding='utf-8-sig')
        df_move_updated.to_csv('./merged_csv/fin/이동내역_fin.csv', index=False, encoding='utf-8-sig')
        
        print("저장 완료!")
        
        return df_visit_updated, df_photo_updated, df_move_updated, name_to_new_id
        
    except Exception as e:
        print(f"오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None

if __name__ == "__main__":
    main()

=== 데이터 로드 ===
방문지: 140555행, 관광사진: 66508행, 이동내역: 140555행
=== NEW_VISIT_AREA_ID 매핑 생성 ===


/var/folders/5y/zrfb8w194h75z3j0v71vttf00000gn/T/ipykernel_70489/4212065480.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
/var/folders/5y/zrfb8w194h75z3j0v71vttf00000gn/T/ipykernel_70489/4212065480.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
/var/folders/5y/zrfb8w194h75z3j0v71vttf00000gn/T/ipykernel_70489/4212065480.py:31: FutureWarning: The behavior o

NEW_ID 1: '(구) 전남도 청사'
NEW_ID 2: '(구)군산 세관 군산근대역사박물관'
NEW_ID 3: '(오오오오) OoOoOH'
NEW_ID 4: '(재단) 인천 교구 천주교회 유지재단'
NEW_ID 5: '(주) 건화 물산(롯데슈퍼체인 상품 공급점)'
NEW_ID 6: '(주) 경인 석유 고속도로 동명(뷰성) 주유소'
NEW_ID 7: '(주) 고속 에너지'
NEW_ID 8: '(주) 남문 소나타 카페'
NEW_ID 9: '(주) 대전 신세계 일로 르노 X 풀 바셋'
NEW_ID 10: '(주) 매머드'
NEW_ID 11: '(주) 서 온 유통 탑 마트 신촌점'
NEW_ID 12: '(주) 신세계 니코르 AK  홍대점'
NEW_ID 13: '(주) 제주는 랑 제주고씨네'
NEW_ID 14: '(주) 주암(순천방향) 주유소'
NEW_ID 15: '(주) 중앙 스포츠'
NEW_ID 16: '(주)해누리'
NEW_ID 17: '01도라'
NEW_ID 18: '033다이닝'
NEW_ID 19: '0415카페'
NEW_ID 20: '1.5닭갈비'
NEW_ID 21: '1004뮤지엄파크'
NEW_ID 22: '1004민박'
NEW_ID 23: '100번지'
NEW_ID 24: '100시트'
NEW_ID 25: '101닭갈비'
NEW_ID 26: '101마켓 전주지점'
NEW_ID 27: '103랩'
NEW_ID 28: '1059-3 Coffee House'
NEW_ID 29: '105소호'
NEW_ID 30: '108하늘계단'
NEW_ID 31: '10E CAFE'
NEW_ID 32: '10전비'
NEW_ID 33: '1158족욕 카페'
NEW_ID 34: '1158흑돼지'
NEW_ID 35: '11고지 습지'
NEW_ID 36: '11고지 휴게소'
NEW_ID 37: '11고지 휴게소 주차장'
NEW_ID 38: '11남매 떡볶이'
NEW_ID 39: '11번 할미 닭발'
NEW_ID 40: '11번가'
NEW_ID 41: '11번지 남산 돈가스 종로타

# 매핑 파일 기반으로 매핑 진행

In [35]:
import os
import pandas as pd

# 매핑 결과 파일 로드
mapping_df = pd.read_csv('./merged_csv/fin/방문지_fin.csv')
mapping_df['TRAVEL_ID'] = mapping_df['TRAVEL_ID'].fillna('').astype(str)
mapping_df['VISIT_AREA_ID'] = mapping_df['VISIT_AREA_ID'].astype(str)

# 매핑 딕셔너리 생성
pair_to_new_id = mapping_df.set_index(['TRAVEL_ID', 'VISIT_AREA_ID'])['NEW_VISIT_AREA_ID'].to_dict()
name_to_new_id = mapping_df.set_index('VISIT_AREA_NM')['NEW_VISIT_AREA_ID'].to_dict()
id_to_new_id = mapping_df.set_index('VISIT_AREA_ID')['NEW_VISIT_AREA_ID'].to_dict()
# 핵심: ID -> 장소명 매핑 (이동내역용)
id_to_name = mapping_df.set_index('VISIT_AREA_ID')['VISIT_AREA_NM'].to_dict()

data_dir = './merged_csv/merged_csv_region/'
region_list = [r for r in os.listdir(data_dir) if not r.startswith('.') and os.path.isdir(os.path.join(data_dir, r))]

for region in region_list:
    print(f"\n=== {region} 처리 시작 ===")
    region_path = os.path.join(data_dir, region)
    region_output_dir = os.path.join(region_path, 'fin')
    os.makedirs(region_output_dir, exist_ok=True)

    # 1️⃣ 방문지 정보 처리
    visit_file = os.path.join(region_path, '방문지_total.csv')
    if os.path.exists(visit_file):
        df_visit = pd.read_csv(visit_file)
        df_visit['TRAVEL_ID'] = df_visit['TRAVEL_ID'].fillna('').astype(str)
        df_visit['VISIT_AREA_ID'] = df_visit['VISIT_AREA_ID'].astype(str)
        
        # 매핑 시도 (우선순위: (TRAVEL_ID, VISIT_AREA_ID) -> VISIT_AREA_ID -> VISIT_AREA_NM)
        new_ids = []
        for _, row in df_visit.iterrows():
            key1 = (row['TRAVEL_ID'], row['VISIT_AREA_ID'])
            key2 = row['VISIT_AREA_ID']
            key3 = row['VISIT_AREA_NM']
            
            if key1 in pair_to_new_id:
                new_ids.append(pair_to_new_id[key1])
            elif key2 in id_to_new_id:
                new_ids.append(id_to_new_id[key2])
            elif key3 in name_to_new_id:
                new_ids.append(name_to_new_id[key3])
            else:
                new_ids.append(None)  # 매핑 실패
        
        df_visit['NEW_VISIT_AREA_ID'] = new_ids
        save_path = os.path.join(region_output_dir, '방문지_fin.csv')
        df_visit.to_csv(save_path, index=False, encoding='utf-8-sig')
        
        unmapped = df_visit['NEW_VISIT_AREA_ID'].isna().sum()
        print(f"{region} 방문지_fin.csv 저장 (shape: {df_visit.shape}, 매핑실패: {unmapped})")

    # 2️⃣ 관광사진 정보 처리
    photo_file = os.path.join(region_path, '관광사진_total.csv')
    if os.path.exists(photo_file):
        df_photo = pd.read_csv(photo_file)
        df_photo['TRAVEL_ID'] = df_photo['TRAVEL_ID'].fillna('').astype(str)
        df_photo['VISIT_AREA_ID'] = df_photo['VISIT_AREA_ID'].astype(str)
        
        new_ids = []
        for _, row in df_photo.iterrows():
            key1 = (row['TRAVEL_ID'], row['VISIT_AREA_ID'])
            key2 = row['VISIT_AREA_ID']
            key3 = row['VISIT_AREA_NM']
            
            if key1 in pair_to_new_id:
                new_ids.append(pair_to_new_id[key1])
            elif key2 in id_to_new_id:
                new_ids.append(id_to_new_id[key2])
            elif key3 in name_to_new_id:
                new_ids.append(name_to_new_id[key3])
            else:
                new_ids.append(None)  # 매핑 실패
        
        df_photo['NEW_VISIT_AREA_ID'] = new_ids
        save_path = os.path.join(region_output_dir, '관광사진_fin.csv')
        df_photo.to_csv(save_path, index=False, encoding='utf-8-sig')
        
        unmapped = df_photo['NEW_VISIT_AREA_ID'].isna().sum()
        print(f"{region} 관광사진_fin.csv 저장 (shape: {df_photo.shape}, 매핑실패: {unmapped})")

    # 3️⃣ 이동내역 정보 처리
    move_file = os.path.join(region_path, 'fin', '이동내역_fin.csv')
    if os.path.exists(move_file):
        df_move = pd.read_csv(move_file)
        df_move['TRAVEL_ID'] = df_move['TRAVEL_ID'].fillna('').astype(str)
        
        start_new_ids = []
        end_new_ids = []
        
        for _, row in df_move.iterrows():
            trip_id = str(int(row['TRIP_ID'])) if pd.notna(row['TRIP_ID']) else None
            
            if pd.notna(row['START_VISIT_AREA_ID']):  # START가 있는 경우 (첫 목적지)
                # TRIP_ID를 이용해서 매핑
                if trip_id and trip_id in id_to_name:
                    start_name = id_to_name[trip_id]
                    start_new_ids.append(name_to_new_id.get(start_name))
                else:
                    start_new_ids.append(None)
                end_new_ids.append(None)  # END는 NaN
                
            elif pd.notna(row['END_VISIT_AREA_ID']):  # END가 있는 경우 (경유지)
                # TRIP_ID를 이용해서 매핑
                if trip_id and trip_id in id_to_name:
                    end_name = id_to_name[trip_id]
                    end_new_ids.append(name_to_new_id.get(end_name))
                else:
                    end_new_ids.append(None)
                start_new_ids.append(None)  # START는 NaN
                
            else:  # 둘 다 없는 경우
                start_new_ids.append(None)
                end_new_ids.append(None)
        
        df_move['NEW_START_VISIT_AREA_ID'] = start_new_ids
        df_move['NEW_END_VISIT_AREA_ID'] = end_new_ids
        
        save_path = os.path.join(region_output_dir, '이동내역_fin.csv')
        df_move.to_csv(save_path, index=False, encoding='utf-8-sig')
        
        start_count = sum(1 for x in start_new_ids if pd.notna(x))
        end_count = sum(1 for x in end_new_ids if pd.notna(x))
        start_unmapped = sum(1 for i, row in df_move.iterrows() if pd.notna(row['START_VISIT_AREA_ID']) and pd.isna(start_new_ids[i]))
        end_unmapped = sum(1 for i, row in df_move.iterrows() if pd.notna(row['END_VISIT_AREA_ID']) and pd.isna(end_new_ids[i]))
        
        print(f"{region} 이동내역_fin.csv 저장 (shape: {df_move.shape})")
        print(f"  - START 매핑: {start_count}개 성공, {start_unmapped}개 실패")
        print(f"  - END 매핑: {end_count}개 성공, {end_unmapped}개 실패")

print("\n=== 모든 지역 처리 완료 ===")


=== 서부권 처리 시작 ===
서부권 방문지_fin.csv 저장 (shape: (31875, 26), 매핑실패: 0)
서부권 관광사진_fin.csv 저장 (shape: (16549, 15), 매핑실패: 0)
서부권 이동내역_fin.csv 저장 (shape: (31875, 15))
  - START 매핑: 2883개 성공, 0개 실패
  - END 매핑: 28992개 성공, 0개 실패

=== 동부권 처리 시작 ===
동부권 방문지_fin.csv 저장 (shape: (32930, 26), 매핑실패: 0)
동부권 관광사진_fin.csv 저장 (shape: (16266, 15), 매핑실패: 0)
동부권 이동내역_fin.csv 저장 (shape: (32930, 15))
  - START 매핑: 2882개 성공, 0개 실패
  - END 매핑: 30048개 성공, 0개 실패

=== 제주도 및 도서지역 처리 시작 ===
제주도 및 도서지역 방문지_fin.csv 저장 (shape: (51596, 26), 매핑실패: 0)
제주도 및 도서지역 관광사진_fin.csv 저장 (shape: (17224, 15), 매핑실패: 0)
제주도 및 도서지역 이동내역_fin.csv 저장 (shape: (51596, 15))
  - START 매핑: 2881개 성공, 0개 실패
  - END 매핑: 48715개 성공, 0개 실패

=== 수도권 처리 시작 ===
수도권 방문지_fin.csv 저장 (shape: (24154, 26), 매핑실패: 0)
수도권 관광사진_fin.csv 저장 (shape: (16469, 15), 매핑실패: 0)
수도권 이동내역_fin.csv 저장 (shape: (24154, 15))
  - START 매핑: 2883개 성공, 0개 실패
  - END 매핑: 21271개 성공, 0개 실패

=== 모든 지역 처리 완료 ===


In [36]:
move_test = pd.read_csv('/Users/idaeho/Documents/GitHub/GNN_Recommend/process_total/merged_csv/merged_csv_region/서부권/fin/이동내역_fin.csv')
move_test.head()

,Unnamed: 0,TRAVEL_ID,TRIP_ID,START_VISIT_AREA_ID,END_VISIT_AREA_ID,START_DT_MIN,END_DT_MIN,MVMN_CD_1,MVMN_CD_2,REGION,DATA_TYPE,NEW_START_VISIT_AREA_ID,NEW_END_VISIT_AREA_ID,START_VISIT_AREA_NM,END_VISIT_AREA_NM
0,0,e_e000005,2304290001,2.304290e+09,NaN,2023-04-29 12:00,NaN,0.0,0.0,서부권,_csv_train,2500.0,NaN,NaN,NaN
1,1,e_e000005,2304290002,NaN,2.304290e+09,NaN,2023-04-29 12:30,1.0,0.0,서부권,_csv_train,NaN,2500.0,NaN,NaN
2,2,e_e000005,2304290003,NaN,2.304290e+09,NaN,2023-04-29 13:30,1.0,0.0,서부권,_csv_train,NaN,28394.0,NaN,NaN
3,3,e_e000005,2304290004,NaN,2.304290e+09,NaN,2023-04-29 16:30,1.0,0.0,서부권,_csv_train,NaN,23161.0,NaN,NaN
4,4,e_e000005,2304290005,NaN,2.304290e+09,NaN,2023-04-29 17:00,5.0,0.0,서부권,_csv_train,NaN,29318.0,NaN,NaN
